In [1]:
import json
import pandas as pd


"""
TODO:
1. Data sudah bener2 post_canonical, tinggal processing untuk di training BERT
2. Tambah new column untuk ini mungkin namanya `post_processed`
"""

def remove_redundant_white_spaces(text: str) -> str:
    from re import sub
    return sub(r'\s+', ' ', text).strip()


file_path = 'C:/wamp64/www/artofproblemsolving/data/data_post_canonical.csv'
df = pd.read_csv(file_path)


In [11]:
def processing_dollar_sign(text: str):
    def extract_text_that_contain_dollar_sign_into_a_list(text: str) -> list:
        """
        Case:
        - "lorem lorem $target 1$ $$target 2 lorem lorem"  -->  ['target 1', '']
        - "lorem lorem $target 1$ $$target 2$$ lorem lorem"  -->  ['target 1', 'target 2']
        - "lorem lorem $target 1$ $$   target 2 $$ lorem lorem"  -->  ['target 1', 'target 2']
        """
        import re

        matches: list = re.findall(r'\$+(.*?)\$+', text)
        for i in range(len(matches)):
            matches[i] = matches[i].strip()

        return matches

    def replace_le_leq_leqslant_with_LESSEQUALTHANSIGN(input_string) -> str:
        import re

        # \le \leq \leqslant
        le_patterns = [r'\\leqslant', r'\\leq', r'\\le']

        for pattern in le_patterns:
            input_string = re.sub(pattern, ' LESSEQUALTHANSIGN ', input_string)

        return remove_redundant_white_spaces(input_string)

    def replace_ge_geq_geqslant_with_GREATEREQUALTHANSIGN(input_string) -> str:
        import re
        
        # \ge \geq \geqslant
        ge_patterns = [r'\\geqslant', r'\\geq', r'\\ge']

        for pattern in ge_patterns:
            input_string = re.sub(pattern, ' GREATEREQUALTHANSIGN ', input_string)

        return remove_redundant_white_spaces(input_string)

    def replace_cdots_ldots_hdots_dots_with_THREEDOTS(input_string):
        import re
        
        # \cdots \ldots \hdots \dots
        dots_patterns = [r'\\cdots', r'\\ldots', r'\\hdots', r'\\dots']
        
        for pattern in dots_patterns:
            input_string = re.sub(pattern, ' THREEDOTS ', input_string)
        
        return remove_redundant_white_spaces(input_string)

    def replace_neq_ne_with_NOTEQUALTOSIGN(input_string):
        import re
        
        # \ne \neq
        ne_patterns = [r'\\neq', r'\\ne']
        
        for pattern in ne_patterns:
            input_string = re.sub(pattern, ' NOTEQUALTOSIGN ', input_string)
        
        return remove_redundant_white_spaces(input_string)

    # \quad \qquad 
    # \text \textrm \text.. ...
    # \mathrm \mathbf \mathsf ...

    # Case LaTex that look like this `\[ ... \]`
    text = text.replace("\[", "$")
    text = text.replace("\]", "$")
    list_dollar = extract_text_that_contain_dollar_sign_into_a_list(text)
    
    for i in range(len(list_dollar)):
        list_dollar[i] = replace_le_leq_leqslant_with_LESSEQUALTHANSIGN(list_dollar[i])
        list_dollar[i] = replace_ge_geq_geqslant_with_GREATEREQUALTHANSIGN(list_dollar[i])
        list_dollar[i] = replace_cdots_ldots_hdots_dots_with_THREEDOTS(list_dollar[i])
        list_dollar[i] = replace_neq_ne_with_NOTEQUALTOSIGN(list_dollar[i])
        
        # list_dollar[i] = list_dollar[i].replace("\\", " ")
        list_dollar[i] = list_dollar[i].replace(">", " GREATERTHANSIGN ")
        list_dollar[i] = list_dollar[i].replace("<", " LESSTHANSIGN ")
        list_dollar[i] = list_dollar[i].replace("+", " PLUS_OR_POSITIVESIGN ")
        list_dollar[i] = list_dollar[i].replace("-", " MINUS_OR_NEGATIVESIGN ")
        list_dollar[i] = list_dollar[i].replace("=", " EQUALTOSIGN ")
        list_dollar[i] = list_dollar[i].replace("!", " FACTORIALSIGN ")
        
    return list_dollar
    

In [17]:
from random import randint
n = randint(0, 10000)
test_ = df.iloc[n]['post_canonical']
link_ = df.iloc[n]['link']

print(n)
print(link_)
print(test_)
print(processing_dollar_sign(test_))


9915
https://artofproblemsolving.com/community/c3724_2003_tournament_of_towns
Prove that every positive integer can be represented in the form \[3^{u_1} \ldots 2^{v_1} + 3^{u_2} \ldots 2^{v_2} + \ldots + 3^{u_k} \ldots 2^{v_k}\] with integers $u_1, u_2, \ldots , u_k, v_1, \ldots, v_k$ such that $u_1 > u_2 >\ldots > u_k\ge 0$ and $0 \le v_1 < v_2 <\ldots < v_k$.
['3^{u_1} THREEDOTS 2^{v_1}  PLUS_OR_POSITIVESIGN  3^{u_2} THREEDOTS 2^{v_2}  PLUS_OR_POSITIVESIGN  THREEDOTS  PLUS_OR_POSITIVESIGN  3^{u_k} THREEDOTS 2^{v_k}', 'u_1, u_2, THREEDOTS , u_k, v_1, THREEDOTS , v_k', 'u_1  GREATERTHANSIGN  u_2  GREATERTHANSIGN  THREEDOTS  GREATERTHANSIGN  u_k GREATEREQUALTHANSIGN 0', '0 LESSEQUALTHANSIGN v_1  LESSTHANSIGN  v_2  LESSTHANSIGN  THREEDOTS  LESSTHANSIGN  v_k']


In [ ]:
for i in range(df.shape[0]):
    df.iloc[i]['']
    ...

In [25]:
"""
NOTE:
Jangan lupa `$...$ hilangin dolar`
Extrak yang diapit dolar (ada yag `$$...$$`) atau lebih dolarnya

1. Consider `\legslant` and `\leg`
"""

# >
biggerthan = "biggerthan"
# <
smallerthan = "smallerthan"
# +
plus_or_positive = "plus_or_positive"
# -
minus_or_negative = "minus_or_negative"
# / == \frac{}{}

# :

# =
equal = "equal"
# !
factorial = "factorial"

# ^
superscript_variable = "superscript_variable"
# _
subscript_variable = "subscript_variable"
# ^_
combined_supersubscript_variable = "combined_supersubscript_variable"

# ABC --> "uppercase_variable uppercase_variable uppercase_variable"
uppercase_variable = "uppercase_variable"
# n, x, y --> lowercase_variable
lowercase_variable = "lowercase_variable"
